Title: Examining Puerto Rico’s Forest Transition with Tree Cover Prediction Models `puerto-rico-forest-transition`

Author(s): Mattie Gisselbeck

**Abstract**

Methods
- Random Forest Model
- Exploratory Regression
- Hot Spot Analysis
- Generalized Linear Regression (GLR)
- Spatial Autocorrelation (Global Moran's I)

In [15]:
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
import numpy as np


from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor


import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd

In [ ]:
# Define Paths

# Methods

1.1 Exploratory Regression `arcpy`

Exploratory Regression was performed to analyze the relationship between forest cover and variables (precipitation, soil productivity, population, protection level, poverty, slope).

In [ ]:
# Perform Exploratory Regression
arcpy.stats.ExploratoryRegression(
    Input_Features="PuertoRico_1km",
    Dependent_Variable="ForestCover",
    Candidate_Explanatory_Variables="Precipitation;SoilProductivity;Population;PercentPoverty;Slope",
    Weights_Matrix_File=None,
    Output_Report_File=r"\\Mac\Home\Desktop\OutputReportFile.txt",
    Output_Results_Table=r"\\Mac\Home\Desktop\OutputResultsTable",
    Maximum_Number_of_Explanatory_Variables=5,
    Minimum_Number_of_Explanatory_Variables=1,
    Minimum_Acceptable_Adj_R_Squared=0.5,
    Maximum_Coefficient_p_value_Cutoff=0.05,
    Maximum_VIF_Value_Cutoff=7.5,
    Minimum_Acceptable_Jarque_Bera_p_value=0.1,
    Minimum_Acceptable_Spatial_Autocorrelation_p_value=0.1
)

1.1.r Results

In [5]:
# Print Output Report File
with open('/Users/mattiegisselbeck/Desktop/OutputReportFile.txt', 'r') as file:
    content = file.read()
    print(content)

******************************************************************************
Choose 1 of 5 Summary
         Highest Adjusted R-Squared Results        
AdjR2     AICc   JB K(BP)  VIF   SA   Model              
 0.60 73140.14 0.00  0.30 1.00 0.00  +SLOPE***           
 0.47 75459.77 0.00  0.59 1.00 0.00  -SOILPRODUCTIVITY***
 0.24 78542.33 0.00  0.00 1.00 0.00  +PRECIPITATION***   
       Passing Models       
AdjR2 AICc JB K(BP) VIF SA   Model
******************************************************************************
Choose 2 of 5 Summary
              Highest Adjusted R-Squared Results              
AdjR2     AICc   JB K(BP)  VIF   SA   Model                         
 0.63 72350.21 0.00  0.00 2.05 0.00  -SOILPRODUCTIVITY***  +SLOPE***
 0.63 72408.17 0.00  0.00 1.18 0.00  -POPULATION***  +SLOPE***      
 0.62 72600.18 0.00  0.00 1.25 0.00  +PRECIPITATION***  +SLOPE***   
       Passing Models       
AdjR2 AICc JB K(BP) VIF SA   Model
***********************************************

1.2 Spatial Autocorrelation (Global Moran's I) `arcpy`

Spatial Autocorrelation (Global Moran's) was performed to assess the spatial autocorrelation of forest cover, taking into account nearest neighbor interactions. In this context, obtaining a "random" outcome is desirable, as it indicates that the residuals are uncorrelated. Conversely, a "random" result suggests that forest cover in one region of Puerto Rico is dependent on the surrounding forest cover.

In [ ]:
# Run Spatial Autocorrelation
arcpy.stats.SpatialAutocorrelation(
    Input_Feature_Class="PuertoRico_1km",
    Input_Field="ForestCover",
    Conceptualization_of_Spatial_Relationships="INVERSE_DISTANCE",
    Distance_Method="EUCLIDEAN_DISTANCE",
    Standardization="ROW",
    Distance_Band_or_Threshold_Distance=None,
    Weights_Matrix_File=None,
    number_of_neighbors=None
)

1.2.r Results

Global Moran's I Summary

| Moran's Index | Expected Index | Variance | Z-Score    | P-Value  |
|---------------|----------------|----------|------------|----------|
| 0.495739      | -0.000115      | 0.000002 | 388.780561 | 0.000000 |

1.3 Hot Spot Analysis `scipy`

Hot Spot Analysis (Getis-Ord Gi) was used to identify if local areas of forest cover is significantly above or below the island's mean. The results of the Hot Spot Analysis are shown below in Figure 6. The areas marked as hot spots have a greater forest cover than the island's average whereas the cold spots have less forest cover than average.

In [15]:
def hot_spot_analysis(file_path):

    # Read the SHP into the Dataframe
    puerto_rico_1km_df = gpd.read_file(file_path)

    # Extract the Forest Cover Values
    forest_cover = puerto_rico_1km_df['Forest2000'].values

    # Calculate the Z-Scores for Each Forest Cover Value
    z_score = stats.zscore(forest_cover)

    # Calculate the P-Values for Each Z-Score using a Two-Tailed Test
    p_value = stats.norm.sf(abs(z_score)) * 2

    # Create a Dataframe with the Forest Cover Values, Z-Scores, and P-Values
    forest_spots_df = pd.DataFrame({"Forest Cover": forest_cover, "Z-Score": z_score, "P-Value": p_value})

    # Add a Column Indicating Whether Each Observation is a Hot Spot (1) or a Cold Spot (-1)
    forest_spots_df["Spot Type"] = np.where(forest_spots_df["Z-Score"] > 0, 1, -1)

    # Save Results
    forest_spots_df.to_csv("PuertoRico_1km_Spots.csv", index=False)

    df = pd.read_csv('PuertoRico_1km_Spots.csv')
    print(df.head())

1.3.r Results

In [16]:
# Perform Hot Spot Analysis
hot_spot_analysis('/Users/mattiegisselbeck/Desktop/PuertoRico_1km.shp')

   Forest Cover   Z-Score   P-Value  Spot Type
0     59.797578  0.285049  0.775607          1
1     60.943772  0.322997  0.746698          1
2     37.321799 -0.459083  0.646174         -1
3     26.346021 -0.822471  0.410809         -1
4     41.185986 -0.331147  0.740533         -1


1.4 Random Forest Model `scikit-learn`

Insert Text Here

In [12]:
def random_forest_model(file_path):

    # Read the CSV file into a DataFrame
    puerto_rico_1km_df = pd.read_csv(file_path)

    # Define Columns
    forest_transition_columns = ['ObjectID', 'Precipitation', 'SoilProductivity', 'Population',
           'ForestCover', 'TreeGain', 'TreeLoss', 'Friction',
           'ProtectionLevel', 'Elevation', 'PctPoverty', 'Slope']

    # Extract Columns and Drop Any Rows with Missing Values
    forest_transition_df = puerto_rico_1km_df[forest_transition_columns].dropna()

    # Split the Dataset into the Features (X) and the Target (y)
    X = forest_transition_df.drop("ObjectID", axis = 1).drop("ForestCover", axis = 1)
    y = forest_transition_df["ForestCover"]

    # Split the Dataset into Training and Testing Sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Fit a Random Forest Regression Model to the Training Set
    random_forest_model = RandomForestRegressor(n_estimators=100, random_state=42)
    random_forest_model.fit(X_train, y_train)

    # Make Predictions on the Test Set and Evaluate the Model
    y_test_predicted = random_forest_model.predict(X_test)
    mse = mean_squared_error(y_test, y_test_predicted)
    r2 = r2_score(y_test, y_test_predicted)

    # Print the Model Evaluation Metrics
    print("Random Forest Model Evaluation")
    print(f"MSE: {mse:.2f}")
    print(f"r2: {r2:.2f}")

1.4.r Results

In [14]:
# Run Random Forest Model
random_forest_model("/Users/mattiegisselbeck/Documents/GitHub/puerto-rico-forest-transition/data/PR_1km.csv")

Random Forest Model Evaluation
MSE: 157.36
r2: 0.83


1.5 Generalized Linear Regression (GLR) `statsmodels.api` `scikit-learn`

A Generalized Linear Regression was performed using `statsmodels.api` to examine the relationship between the independent variables and the dependent variable.

In [16]:
# Set Dataframe
puerto_rico_1km_df = gpd.read_file('/Users/mattiegisselbeck/Desktop/PuertoRico_1km.shp')

# Define Independent Variables
X_1km_columns = ['Precip', 'Soil', 'Settlement',
       'Protected', 'Elevation', 'Friction', 'PctPov', 'Slope']

X_1km = puerto_rico_1km_df[X_1km_columns]

# Define Dependent Variable
y_1km = puerto_rico_1km_df['Forest2000']

# Fit GLR Model for 1km Fishnet Dataset
X_1km = sm.add_constant(X_1km)
model_1km = sm.GLM(y_1km, X_1km, family=sm.families.Gaussian())
result_1km = model_1km.fit()

1.5.r Results

In [17]:
result_1km.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:             Forest2000   No. Observations:                 8682
Model:                            GLM   Df Residuals:                     8673
Model Family:                Gaussian   Df Model:                            8
Link Function:               identity   Scale:                          285.39
Method:                          IRLS   Log-Likelihood:                -36858.
Date:                Thu, 27 Apr 2023   Deviance:                   2.4752e+06
Time:                        11:32:12   Pearson chi2:                 2.48e+06
No. Iterations:                     3   Pseudo R-squ. (CS):             0.8889
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.2188      1.619      5.695      0.000       6.046      12.391
Precip         0.0174      0.001     26.557      0.000       0.016       0.019
Soil          -0.0439      0.002    -21.073      0.000      -0.048      -0.040
Settlement    -0.1160      0.004    -27.981      0.000      -0.124      -0.108
Protected      4.6288      0.280     16.541      0.000       4.080       5.177
Elevation     -0.0097      0.001     -7.347      0.000      -0.012      -0.007
Friction       0.0003      0.000      2.478      0.013    6.72e-05       0.001
PctPov         0.0680      0.016      4.202      0.000       0.036       0.100
Slope          2.5449      0.058     44.006      0.000       2.432       2.658
==============================================================================
"""

1.6 Linear Regression, Lasso, Ridge, Decision Tree Regression, Random Forest Regression, and XGBoost Regression `scikit-learn` `xgboost`

A five-fold cross-validation was used to determine the performance of each model on the forest transition training set. Each model was fitted on the entire training set and was evaluated by its performance on the test set. The results are shown in 1.6.r.

In [11]:
# Set Dataframe
puerto_rico_1km_df = pd.read_csv('/Users/mattiegisselbeck/Documents/GitHub/puerto-rico-forest-transition/data/PR_1km.csv')

# Define Columns
forest_transition_columns = ['ObjectID', 'Precipitation', 'SoilProductivity', 'Population',
           'ForestCover', 'TreeGain', 'TreeLoss', 'Friction',
           'ProtectionLevel', 'Elevation', 'PctPoverty', 'Slope']

# Extract Columns and Drop Any Rows with Missing Values
forest_transition_df = puerto_rico_1km_df[forest_transition_columns].dropna()

# Split the Dataset into the Features (X) and the Target (y)
X = forest_transition_df.drop("ObjectID", axis = 1).drop("ForestCover", axis = 1)
y = forest_transition_df["ForestCover"]

# Split the Dataset into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Models
models = [
    LinearRegression(),
    Lasso(),
    Ridge(),
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    XGBRegressor()
]

# Evaluate the Models using Cross-Validation
for model in models:
    scores = cross_val_score(model, X_train, y_train, cv=5)
    print(f"{type(model).__name__} r2: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

    # Fit the Model(s) and Evaluate on the Test Set
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    print(f"{type(model).__name__} Test Accuracy: {r2:.2f}")

LinearRegression r2: 0.73 (+/- 0.03)
LinearRegression Test Accuracy: 0.72
Lasso r2: 0.70 (+/- 0.04)
Lasso Test Accuracy: 0.69
Ridge r2: 0.73 (+/- 0.03)
Ridge Test Accuracy: 0.72
DecisionTreeRegressor r2: 0.62 (+/- 0.05)
DecisionTreeRegressor Test Accuracy: 0.65
RandomForestRegressor r2: 0.82 (+/- 0.02)
RandomForestRegressor Test Accuracy: 0.83
XGBRegressor r2: 0.82 (+/- 0.02)
XGBRegressor Test Accuracy: 0.83


1.6.r Results

In [20]:
print("Random Forest Model Evaluation")
print(f"r2: {r2:.2f}")

Random Forest Model Evaluation
r2: 0.83
